### Housing Price Prediction With Multiple Linear Regression

In this end-to-end project, I will provide an overview of the multiple linear regression model fitting process, from gathering the data and selecting independent variables with high predictive powers to training and testing the model and calculating the Mean Absolute Error.

A detailed walkthrough can be found on my website at: [kostya.io](https://kostya.io/portfolio/housing-price-prediction-with-multiple-linear-regression)

In [1]:
# Load packages

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import requests
from statistics import mean
from matplotlib.cbook import boxplot_stats
import warnings
from sklearn import linear_model
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore")

In [2]:
# Load the transaction dataset
df = pd.read_csv('Santa Clara County SFR Transaction 30 Days.csv')

In [3]:
list(df)

['S',
 'MLS #',
 'Street Address',
 'Price',
 'DOM',
 'Beds Total',
 'Sq Ft Total',
 'Bths',
 'Lot Size',
 'Postal City',
 'Property Sub Type',
 'Age']

In [4]:
 # Drop columns that I am definitely not going to use (because they don't contain enough useful information)
df = df.drop(['S',
              'MLS #',
              'DOM',
              'Property Sub Type'], axis=1)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 872 entries, 0 to 871
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Street Address  872 non-null    object
 1   Price           872 non-null    object
 2   Beds Total      872 non-null    int64 
 3   Sq Ft Total     872 non-null    object
 4   Bths            872 non-null    object
 5   Lot Size        872 non-null    object
 6   Postal City     871 non-null    object
 7   Age             872 non-null    int64 
dtypes: int64(2), object(6)
memory usage: 54.6+ KB


In [6]:
# Clean 'Lot Size' column
acres = ' Acres'
SqFt = ' Lot SqFt'

for i in range(0, len(df['Lot Size'])):
    if acres in df['Lot Size'][i]:
        df['Lot Size'][i] = pd.to_numeric(df['Lot Size'][i].replace(acres, '')) * 43560
        
for j in range(0, len(df['Lot Size'])):
    if isinstance(df['Lot Size'][j], str):
        df['Lot Size'][j] = df['Lot Size'][j].replace(SqFt, '')
        df['Lot Size'][j] = df['Lot Size'][j].replace(',', '')
        
df['Lot Size'] = pd.to_numeric(df['Lot Size'])

In [7]:
# Clean 'Sq Ft Total' column
    
for i in range(0, len(df['Sq Ft Total'])):
        df['Sq Ft Total'][i] = df['Sq Ft Total'][i].replace(',', '')
        
df['Sq Ft Total'] = pd.to_numeric(df['Sq Ft Total'])

In [8]:
# Clean 'Price' column
    
for i in range(0, len(df['Price'])):
        df['Price'][i] = df['Price'][i].replace(',', '')
        df['Price'][i] = df['Price'][i].replace('$', '')
df['Price'] = pd.to_numeric(df['Price'])

In [9]:
# Create Full Bath & Half Bath columns

df['Full Bath'] = 0
df['Half Bath'] = 0

for i in range(0,len(df)):
    df['Full Bath'][i] = df['Bths'][i][:1]
    df['Half Bath'][i] = df['Bths'][i][-1:] 


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 872 entries, 0 to 871
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Street Address  872 non-null    object 
 1   Price           872 non-null    int64  
 2   Beds Total      872 non-null    int64  
 3   Sq Ft Total     872 non-null    int64  
 4   Bths            872 non-null    object 
 5   Lot Size        872 non-null    float64
 6   Postal City     871 non-null    object 
 7   Age             872 non-null    int64  
 8   Full Bath       872 non-null    int64  
 9   Half Bath       872 non-null    int64  
dtypes: float64(1), int64(6), object(3)
memory usage: 68.2+ KB


In [11]:
df = df.drop(['Bths'], axis=1)

In [12]:
df.head()

,Street Address,Price,Beds Total,Sq Ft Total,Lot Size,Postal City,Age,Full Bath,Half Bath
0,5891 Pontius Court,1600000,4,1348,6000.0,San Jose,51,2,0
1,19600 Caraway Place,1390000,4,2501,3717.0,Morgan Hill,6,2,1
2,1440 Mount Diablo Drive,985000,3,1000,5000.0,San Jose,64,2,0
3,555 N 10th Street,960000,3,1200,9520.0,San Jose,114,1,0
4,27729 Briones Court,9335000,5,4827,43560.0,Los Altos Hills,7,4,1


### Google Maps API: Geocoding

Create and populate Latitude / Longitude columns and populate them with corresponding information for each property address

In [ ]:
def google_geocode(geocode_location):
    gmaps = googlemaps.Client(key='< ADD YOU API KEY >')
    geocode_result = gmaps.geocode(geocode_location)
    return geocode_result

In [ ]:
df['State'] = 'CA'
df['lat'], df['lon'] = 0.0, 0.0

for i in range(0, len(df)):
    geocode_location = "{}, {}, {}".format(df['Street Address'][i],df['Postal City'][i],df['State'][i])
    geo_res = google_geocode(geocode_location)
    df['lat'][i] = float(geo_res[0]['geometry']['location']['lat'])
    df['lon'][i] = float(geo_res[0]['geometry']['location']['lng'])

### US Census Geocoder (Geographies)

Get census block ID's from US Census API.

In [ ]:
def get_geography(x_coordinate, y_coordinate):

    # api-endpoint
    URL = "https://geocoding.geo.census.gov/geocoder/geographies/coordinates?benchmark=2020&format=json&vintage=Census2020_Census2020"

    # defining a params dict for the parameters to be sent to the API
    PARAMS = {'x': x_coordinate,
              'y': y_coordinate,
             'key': '< ADD YOU API KEY >'}
  
    # sending get request and saving the response as response object
    geography_r = requests.get(url = URL, params = PARAMS)

    geography_data = geography_r.json()
    
    return geography_data

In [ ]:
for i in range(0, len(df)): 
    x_coordinate = df['lon'][i]
    y_coordinate = df['lat'][i]
    geo = get_geography(x_coordinate, y_coordinate)
    df['Tract'][i] = geo['result']['geographies']['Census Tracts'][0]['TRACT']
    df['County'][i] = geo['result']['geographies']['Counties'][0]['BASENAME']

### US Census API (Total Population / Median Income)

Get Total Population and Median Household Income statistics for each census block.

In [ ]:
def population_income(tract, county):

    # api-endpoint
    URL = "https://api.census.gov/data/2020/acs/acs5"

    # defining a params dict for the parameters to be sent to the API
    PARAMS = {'key': '< ADD YOU API KEY >',
              'in': county,
              'get': 'B19013_001E,B01003_001E',
              'for': tract
             }
    
    # sending get request and saving the response as response object
    r = requests.get(url = URL, params = PARAMS)

    population_income_data = r.json()
    
    return population_income_data

In [ ]:
df['Total Population'] = 0
df['Median Income'] = 0

for i in range(0, len(df)):
    if df['County'][i] == 'Santa Clara':
        county = 'state:06%20county:085'
    elif df['County'][i] == 'San Francisco':
        county = 'state:06%20county:075'
    elif df['County'][i] == 'San Mateo':
        county = 'state:06%20county:081'
        
    tract = "tract:{}".format(df['Tract'][i])
    
    pop_inc = population_income(tract, county)
       
    df['Total Population'][i] = pop_inc[1][1]
    df['Median Income'][i] = pop_inc[1][0]

### Walk Score API

Get the walkability and bikeability score for each property location.

In [ ]:
df['walk_score'], df['bike_score'], df['transit_score'] = 0, 0, 0

def walk_score(walkscore_parameters):
    walkscore_url = "https://api.walkscore.com/score"
    walkscore_r = requests.get(url = walkscore_url, params = walkscore_parameters)
    walkscore_data = walkscore_r.json()
    return walkscore_data

In [ ]:
for i in range(0, len(df)):
    walkscore_parameters = {'address': "{},{},{}".format(df['Street Address'][i], df['Postal City'][i], df['State'][i]),
                            'lat': df['lat'][i],
                            'lon': df['lon'][i],
                            'wsapikey' : '< ADD YOU API KEY >',
                            'transit': '1',
                            'bike': '1',
                            'format': 'json',
                            
                           }
    walk_score_data = walk_score(walkscore_parameters)
    df['walk_score'][i] = walk_score_data['walkscore']
    df['bike_score'][i] = walk_score_data['bike']['score']

### Get Comps

Get comparable properties sales prices: 10 most recent house sales with the same number of bedrooms, similar count of bathrooms, located in the same areas with the subject property.

Calculate the average $/ SF price based on the 10 comps.

In [ ]:
def getComps(latitude, longitude, bedrooms, baths):

    url = "https://realty-in-us.p.rapidapi.com/properties/v2/list-sold"

    querystring = {
        "offset":"0",
        "limit":"10",
        "sort":"sold_date",
        "prop_type":"single_family",
        "baths_min": baths,
        "beds_min": bedrooms - 1,
        "lat_max": latitude + 0.006,
        "lat_min": latitude - 0.006,
        "lng_max": longitude + 0.006,
        "lng_min": longitude - 0.006}

    headers = {
        "X-RapidAPI-Key": "< ADD YOU API KEY >",
        "X-RapidAPI-Host": "realty-in-us.p.rapidapi.com"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)

    data = response.json()

    sum = 0
    for i in range(0, len(data['properties'])):
        sum += data['properties'][i]['price'] / data['properties'][i]['building_size']['size']
    average_price = sum / len(data['properties'])
    return average_price

In [ ]:
for i in range(833, len(df)):
    latitude = df['lat'][i]
    longitude = df['lon'][i]
    bedrooms = df['Beds Total'][i]
    baths = df['Fbath'][i]
    
    comps_avg = getComps(latitude, longitude, bedrooms, baths)
    
    df['comps_avg'][i] = comps_avg
    print(i, df['comps_avg'][i])

### Get School Ratings

Get the average school rating (K-12) for each property location.

In [ ]:
def getSchoolRating(latitude, longitude):
    url = "https://realty-in-us.p.rapidapi.com/schools/list-nearby"

    querystring = {
        "lon":longitude,
        "lat":latitude
    }

    headers = {
        "X-RapidAPI-Key": "4762e71129mshb01aaab56476586p15ff98jsn4667ebc334be",
        "X-RapidAPI-Host": "realty-in-us.p.rapidapi.com"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)

    data = response.json()

    data['schools'][0]['ratings']

    list = []
    for i in range(0, len(data['schools'])):
        if (data['schools'][i]['ratings']['great_schools_rating']) is not None:
            list.append(int(data['schools'][i]['ratings']['great_schools_rating']))
    average_rating = mean(list)    
    return average_rating

In [ ]:
for i in range(0, len(df)):
    latitude = df['lat'][i]
    longitude = df['lon'][i]
    
    rating = getSchoolRating(latitude, longitude)
    
    df['school_rating'][i] = rating
    
    print(i, df['school_rating'][i])

### Data Cleaning and Feature Selection

In [13]:
df = pd.read_csv('mls_dataset.csv')

In [14]:
# Drop non-numerical variable
df = df.drop(['Street Address', 'Postal City', 'County', 'State', 'Lat', 'Lon', 'Tract'], axis=1)

In [15]:
df

,Price,Beds Total,Sq Ft Total,Full Bath,Half Bath,Lot Size,Age,Total Population,Median Income,Walk Score,Bike Score,Comps,School Rating
0,1100000,5,2590,3,0,7094,42,3650,75915,24,24,360,5.2
1,1720000,5,4510,4,1,24430,45,5102,139157,11,3,381,5.6
2,1750000,6,4357,4,0,48787,40,3837,141538,0,0,402,3.8
3,1250000,4,2489,3,0,4355,5,7176,147900,4,48,440,5.2
4,970000,3,2028,2,1,4764,7,7176,147900,17,51,443,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
864,4195000,2,1031,2,0,13012,74,4699,250001,44,87,2094,8.6
865,2710000,3,1240,2,0,7072,66,5339,142692,73,78,2102,8.7
866,19000000,5,8958,6,2,50965,2,4585,250001,1,14,2121,8.4
867,4610000,3,1885,2,0,5000,84,3496,250001,70,98,2123,8.0


In [ ]:
# Create a pairplot to check relationships between various variable pairs
sns.pairplot(df)

In [ ]:
# Create histograms to check distribution of values in each column
df.hist()

In [ ]:
# Create correlation matrix to check Pearson's correlation coefficients for each pair of variables
fig, ax = plt.subplots(figsize = (18,10))
sns.heatmap(df.corr(), annot = True, annot_kws={'size':12})

In [ ]:
# Select columns with higher correlation coefficients
df = df[['Sq Ft Total', 'Comps', 'Full Bath', 'Median Income', 'Price', 'School Rating']]

In [ ]:
# Check skewness coefficients for each variable
df.agg(['skew']).T

In [ ]:
# Use log transformation to reduce skewness for select features
vars = ['Sq Ft Total', 'Comps', 'Full Bath', 'Median Income', 'Price']
for i in vars:
    df[i] = np.log(df[i])

In [ ]:
# Create a boxplot chart to see destribution of values in the 'Price' columns (there are obvious outliers above and below whiskers)
df.boxplot(column=['Price'])

In [ ]:
# Get boxplot stats
stats = boxplot_stats(df['Price'].values)

# Remove records with 'Price per Area' values above and beyond upper and lower whiskers respectively.
df.drop(df[(df['Price'] > stats[0]['whishi']) | (df['Price'] < stats[0]['whislo'])].index, inplace = True)

In [ ]:
df.reset_index(drop=True)

In [ ]:
# Use stats model to fit the regression model

X = df[['Sq Ft Total', 'Comps', 'Median Income', 'School Rating', 'Full Bath']]
y = df['Price']
 
# with sklearn
regr = linear_model.LinearRegression()
regr.fit(X, y)

# with statsmodels
X = sm.add_constant(X) # adding a constant
 
model = sm.OLS(y, X).fit()
predictions = model.predict(X) 
 
print_model = model.summary()
print(print_model)


In [ ]:
# Create train and test datasets, train the model, and predict values for properties in the test dataset
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

model = LinearRegression()
model.fit(x_train, y_train)

y_predict = model.predict(x_test)

y_predict = np.exp(y_predict)

y_test = np.exp(y_test)

In [ ]:
# Calculate the absolute errors
errors = abs(y_predict - y_test)

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)

# Calculate and display accuracy
accuracy = 100 - np.mean(mape)

# Print out the mean absolute error (mae) and mean absolute percentage error
print('Mean Absolute Error: $',round(np.mean(errors)))
print('MAPE: ', round(np.mean(mape), 3), '%.',)
print('Accuracy:', round(accuracy, 3), '%.', '(1 - MAPE)')


